In [1]:
import gym 
import numpy as np

from function_approximators.function_approximators import NeuralNetwork, LinearModel
from train_utils import train

from agents.agents import DQNAgent, LinearAgent

RENDER = False
env = gym.make("CartPole-v1")

In [2]:
function_approximators = [NeuralNetwork, LinearModel]
agents = [DQNAgent, LinearAgent]

In [15]:

# for lr in np.linspace(0.001,0.07,20):

returns = []
n_seeds=10

for i in range(n_seeds):

    print(f"\n Run: {i+1} \n")

    # print(f"Learning rate: {lr}")
    # Linear Config
    CONFIG = {
        "episode_length": 200,
        "max_timesteps": 30000,
        "max_time": 30 * 60,
        "eval_freq": 2000, 
        "eval_episodes": 10,
        "learning_rate": 0.035,
        "target_update_freq": 50,
        "batch_size": 32,
        "gamma": 0.99,
        "buffer_capacity": int(1e7),
        "plot_loss": False,
        "epsilon": 1,
        "max_steps": 200,
        "poly_degree": 2,
        "max_deduct": 0.97,
        "decay": 0.1,
    }
    r, _ = train(env, 
            CONFIG, 
            fa=function_approximators[1], 
            agent = agents[1], 
            render=RENDER)
    env.close()
    returns.append(r[-1])

print(f"Mean returns: {np.mean(returns)}")
print(f"Std returns: {np.std(returns)}")

00:15<00:09, 1041.91it/s]Evaluation at timestep 20020 returned a mean returns of 168.4
Learning rate - 0.02862674281590309
 73%|███████▎  | 22022/30000 [00:16<00:05, 1417.65it/s]Evaluation at timestep 22022 returned a mean returns of 149.5
Learning rate - 0.028057070633866615
 81%|████████  | 24314/30000 [00:19<00:06, 812.71it/s] Evaluation at timestep 24154 returned a mean returns of 173.5
Learning rate - 0.02749873492825267
 88%|████████▊ | 26313/30000 [00:20<00:03, 923.54it/s] Evaluation at timestep 26113 returned a mean returns of 196.4
Learning rate - 0.02695151010318044
 94%|█████████▍| 28170/30000 [00:22<00:01, 946.47it/s] Evaluation at timestep 28010 returned a mean returns of 177.7
Learning rate - 0.026681995002148635
30134it [00:24, 1248.95it/s]
  1%|          | 166/30000 [00:00<00:18, 1574.56it/s]Evaluation at timestep 30134 returned a mean returns of 195.6
Learning rate - 0.02588951306858982

 Run: 3 

  7%|▋         | 2110/30000 [00:01<00:31, 885.62it/s] Evaluation at time